In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


DATA_DIR = Path("../cl/raw")


In [ ]:
files = list(DATA_DIR.glob("*.csv.zst"))
print(files)

In [ ]:
df = pd.read_csv(files[0])
df

In [ ]:
symbols = pd.DataFrame(df['symbol'].unique(), columns=['symbol'])
instrument_ids = pd.DataFrame(df['instrument_id'].unique(), columns=['instrument_id'])
symbols_and_ids = pd.merge(df[['symbol', 'instrument_id']].drop_duplicates(), symbols, on='symbol').sort_values(by='symbol').reset_index(drop=True)
symbols_and_ids

In [ ]:
month_codes = "FGHJKMNQUVXZ"
pattern = rf'^CL[{month_codes}]\d$'
cl = df[df['symbol'].str.match(pattern)].copy()
cl = cl.sort_values(by=['ts_event', 'symbol']).reset_index(drop=True)
cl[["ts_event", "symbol", "close", "volume"]]
cl.to_csv(DATA_DIR.parent / "processed" / "cl_raw.csv", index=False)

In [ ]:
rolled_raw = cl.loc[cl.groupby('ts_event')["volume"].idxmax()].reset_index(drop=True)[["ts_event", "symbol", "close", "volume"]]
rolled_raw[["ts_event", "symbol", "volume"]]


In [ ]:
mask = pd.DataFrame()
mask["flicker"] = (rolled_raw["symbol"] != rolled_raw["symbol"].shift(1)) & (rolled_raw["symbol"] != rolled_raw["symbol"].shift(-1))
mask["switch"] = (rolled_raw["symbol"] != rolled_raw["symbol"].shift(-1)) #| (rolled_raw["symbol"] != rolled_raw["symbol"].shift(-1))
mask.iat[-1,1] = False
mask



In [ ]:
mask[mask["switch"]].index

In [ ]:
for i in mask[mask["flicker"]].index:
    rolled_raw.loc[i] = cl.loc[cl["ts_event"] == rolled_raw.loc[i, "ts_event"]].nlargest(2, "volume").iloc[1]
rolled_raw[["ts_event", "symbol", "volume", "close"]]


In [ ]:
rolled_raw.to_csv(DATA_DIR.parent / "processed" / "cl_front_month_raw.csv", index=False)

In [ ]:
plt.plot(pd.to_datetime(rolled_raw['ts_event']), rolled_raw['close'], label='CL Front Month Close Prices')
plt.title('WTI Crude Front Month Close Prices')
plt.xlabel('Time Index')
plt.ylabel('Close Price (USD)')
fig = plt.gcf()
fig.set_size_inches(12, 6)
fig.set_dpi(600)
plt.tight_layout()
plt.show()

In [ ]:
panama_rolled = rolled_raw.copy()
panama_rolled

In [ ]:
panama_rolled = rolled_raw.copy()
for i in mask[mask["switch"]].index:
    panama_rolled.loc[:i, "close"] += (panama_rolled.iat[i+1, 2] - panama_rolled.iat[i, 2]).round(2) # type: ignore
panama_rolled["close"] = panama_rolled["close"].round(2)
panama_rolled

In [ ]:
plt.plot(pd.to_datetime(panama_rolled['ts_event']), panama_rolled['close'], label='CL Front Month Close Prices (Panama Adjusted)')
plt.title('WTI Crude Front Month Close Prices (Panama Adjusted)')
plt.xlabel('Time Index')
plt.ylabel('Close Price (USD)')
fig = plt.gcf()
fig.set_size_inches(12, 6)
fig.set_dpi(600)
plt.tight_layout()
plt.show()


In [ ]:
panama_rolled.to_csv(DATA_DIR.parent / "processed" / "cl_front_month_panama_adjusted.csv", index=False)

In [ ]:
proportional_rolled = rolled_raw.copy()
for i in mask[mask["switch"]].index:
    proportional_rolled.loc[:i, "close"] *= (proportional_rolled.iat[i+1, 2] / proportional_rolled.iat[i, 2]) # type: ignore
panama_rolled["close"] = panama_rolled["close"].round(2)

In [ ]:
plt.plot(pd.to_datetime(proportional_rolled['ts_event']), proportional_rolled['close'], label='CL Front Month Close Prices (Proportional Adjusted)')
plt.title('WTI Crude Front Month Close Prices (Proportional Adjusted)')
plt.xlabel('Time Index')
plt.ylabel('Close Price (USD)')
fig = plt.gcf()
fig.set_size_inches(12, 6)
fig.set_dpi(600)
plt.tight_layout()
plt.show()


In [ ]:
proportional_rolled.to_csv(DATA_DIR.parent / "processed" / "cl_front_month_proportional_adjusted.csv", index=False)